In [35]:
from glob import glob
import os
import csv
import pandas as pd
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

In [36]:
tf.reset_default_graph()

files = glob('results/*.csv')
LOG_DIR = 'tensorboard'
emb = []
for FPATH in files:
    print(FPATH)
    NAME = FPATH.split('/')[-1].split('.')[0]
    df = pd.read_csv(FPATH)
    df['vectors'] = df['vectors'].map(eval)
    features = tf.Variable(df['vectors'].tolist(), name=NAME)
    emb.append(features)

results/doc2vec_with_opencorpora.csv
results/doc2vec_phrases.csv
results/doc2vec_light.csv
results/doc2vec_all.csv
results/doc2vec_phrases_light.csv


In [37]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    saver.save(sess, os.path.join(LOG_DIR, 'emb.ckpt'))
    
    config = projector.ProjectorConfig()
    for i, FPATH in enumerate(files):
        embedding = config.embeddings.add()
        embedding.tensor_name = emb[i].name
        metadata_path = '{}.tsv'.format(FPATH.split('/')[-1].split('.')[0])
        print(FPATH)
        pd.read_csv(FPATH).drop(['vectors'], axis=1)\
        .to_csv(os.path.join(LOG_DIR, metadata_path), sep='\t', index=False, escapechar='\\', quoting=csv.QUOTE_NONE)
        embedding.metadata_path = metadata_path
    
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

results/doc2vec_with_opencorpora.csv
results/doc2vec_phrases.csv
results/doc2vec_light.csv
results/doc2vec_all.csv
results/doc2vec_phrases_light.csv
